In [1]:
%load_ext autoreload
%autoreload 2


import sys
import os

from sklearn.linear_model import SGDClassifier, LinearRegression, Lasso, Ridge
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
import seaborn as sn
import random
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from sklearn.manifold import TSNE
import tqdm
import copy
from sklearn.svm import LinearSVC 

from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import TruncatedSVD
import torch
from sklearn.linear_model import SGDClassifier
from rlace import solve_adv_game

from sklearn.svm import LinearSVC

import sklearn
from sklearn.linear_model import LogisticRegression
import random
import pickle
import matplotlib.pyplot as plt
from sklearn import cluster
from sklearn import neural_network
from gensim.models.keyedvectors import Word2VecKeyedVectors
from gensim.models import KeyedVectors
import numpy as np
import warnings
import argparse
from sklearn.neural_network import MLPClassifier
from collections import defaultdict
import scipy
from scipy import stats
from scipy.stats import pearsonr
import pandas as pd
from rlace import init_classifier
from collections import Counter
from sklearn.linear_model import LogisticRegression
import rlace

In [2]:
bios_data_path = "../WWW/rlace/bios/data/bios_data"
bios_encodings_path = "../WWW/rlace/bios/data/encodings/freezed"

data = []
for mode in ["train", "dev", "test"]:
    with open(bios_data_path + "/" + "{}.pickle".format(mode), "rb") as f:
        data.append(pickle.load(f))
        
train,dev,test = data
gender2ind = {"m": 0, "f": 1}
train_profs, dev_profs = np.array([d["p"] for d in train]), np.array([d["p"] for d in dev])
train_gender, dev_gender = np.array([gender2ind[d["g"]] for d in train]), np.array([gender2ind[d["g"]] for d in dev])


data = []
for mode in ["train", "dev", "test"]:
    with open(bios_encodings_path + "/" + "{}_cls.npy".format(mode), "rb") as f:
        data.append(np.load(f))
        
train_x, dev_x, test_x = data
pca = PCA(n_components=300, random_state=0)
pca.fit(train_x)
train_x = pca.transform(train_x)
dev_x = pca.transform(dev_x)
test_x = pca.transform(test_x)

In [3]:
profs_counts = Counter(train_profs)
common_profs = [p for p,c in profs_counts.most_common(10)]
import itertools
pairs = []
for pair in itertools.product(common_profs, repeat=2):
    if (pair[0], pair[1]) not in pairs and (pair[1], pair[0]) not in pairs and pair[0] != pair[1]:
        pairs.append(pair)

In [4]:
random.seed(0)
np.random.seed(0)
idx = list(range(len(pairs)))
random.shuffle(idx)

pairs_sample = [pairs[i] for i in idx[:15]]

# learn projections

In [ ]:
np.random.seed(0)
random.seed(0)

num_iters = 60000
rank=1
optimizer_class = torch.optim.SGD
optimizer_params_P = {"lr": 0.001, "weight_decay": 1e-5, "momentum":0.5}
optimizer_params_predictor = {"lr": 0.001,"weight_decay": 1e-5, "momentum": 0.5}
epsilon = 0.0025 # stop 0.25% from majority acc
batch_size = 256
rank=1
device="cpu"

random.seed(0)
np.random.seed(0)

for pair in tqdm.tqdm_notebook(pairs_sample):
    print(pair)
    with open("acl/data/{}-{}.pickle".format(pair[0], pair[1]), "rb") as f:
        data = pickle.load(f)
        train_x, train_z, train_y = data["train"]
        dev_x, dev_z, dev_y = data["dev"]
        
        output = solve_adv_game(train_x, train_z, dev_x, dev_z, rank=rank, device=device, out_iters=num_iters, optimizer_class=optimizer_class, optimizer_params_P =optimizer_params_P, optimizer_params_predictor=optimizer_params_predictor, epsilon=epsilon,batch_size=batch_size)
        with open("acl/projs/{}-{}.pickle".format(pair[0], pair[1]), "wb") as f:
            pickle.dump(output, f)

In [21]:
P.keys()

dict_keys(['score', 'P_before_svd', 'P'])

## Train gender classifiers

In [36]:
pair2genderclf = defaultdict(list)
pair2professionclf = defaultdict(list)
pair2genderclf_original = defaultdict(list)
pair2professionclf_original = defaultdict(list)

num_clfs = 10

for pair in tqdm.tqdm_notebook(pairs_sample):
    with open("acl/data/{}-{}.pickle".format(pair[0], pair[1]), "rb") as f:
        data = pickle.load(f)
        train_x, train_z, train_y = data["train"]
        dev_x, dev_z, dev_y = data["dev"]
        
    with open("acl/projs/{}-{}.pickle".format(pair[0], pair[1]), "rb") as f:
        P = pickle.load(f)["P"]

    train_x_proj = train_x@P
    pair_str = pair[0]+ "-" + pair[1]
    dev_x_proj = dev_x@P

    for i in range(num_clfs):
        gender_clf = init_classifier()
        gender_clf.fit(train_x_proj, train_z)
        pair2genderclf[pair_str].append(gender_clf)
        
        prof_clf = init_classifier()
        prof_clf.fit(train_x_proj, train_y)
        pair2professionclf[pair_str].append(prof_clf)
        
        gender_clf = init_classifier()
        gender_clf.fit(train_x, train_z)
        pair2genderclf_original[pair_str].append(gender_clf)
        
        prof_clf = init_classifier()
        prof_clf.fit(train_x, train_y)
        pair2professionclf_original[pair_str].append(prof_clf)

/tmp/ipykernel_21743/900876923.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for pair in tqdm.tqdm_notebook(pairs_sample):


  0%|          | 0/15 [00:00<?, ?it/s]

In [37]:
with open("acl/interim/pair2genderclf.pickle", "wb") as f:
    pickle.dump(pair2genderclf, f)
    
with open("acl/interim/pair2profclf.pickle", "wb") as f:
    pickle.dump(pair2professionclf, f)

with open("acl/interim/pair2genderclf_original.pickle", "wb") as f:
    pickle.dump(pair2genderclf_original, f)
    
with open("acl/interim/pair2profclf_original.pickle", "wb") as f:
    pickle.dump(pair2professionclf_original, f)

In [5]:
def load_gender_clf(pair, ind=0, original=False):

    with open("acl/interim/pair2genderclf{}.pickle".format("_original" if original else ""), "rb") as f:
        pair2genderclf = pickle.load(f)
    return pair2genderclf[pair[0]+"-"+pair[1]][ind]
             
def load_prof_clf(pair, ind=0, original=False):

    with open("acl/interim/pair2genderclf{}.pickle".format("_original" if original else ""), "rb") as f:
        pair2profclf = pickle.load(f)
    return pair2profclf[pair[0]+"-"+pair[1]][ind]
                          

# Train main-task classifier, evaluate V-info

In [6]:
def get_entropy(y):
    
    counts = Counter(y)
    fracts = [c/sum(counts.values()) for c in counts.values()]
    return scipy.stats.entropy(fracts)


In [39]:

random.seed(0)
np.random.seed(0)
num_clfs = 5
do_proj=True
epsilons = [0.55, 0.6,0.7,0.8,0.9,0.99]
pair2vinfo_soft_before = defaultdict(dict)
pair2vinfo_hard_before = defaultdict(dict)
pair2vinfo_soft_after = defaultdict(dict)
pair2vinfo_hard_after = defaultdict(dict)


for pair in pairs_sample[:]:
    
        with open("acl/data/{}-{}.pickle".format(pair[0], pair[1]), "rb") as f:
            data = pickle.load(f)
            train_x, train_z, train_y = data["train"]
            dev_x, dev_z, dev_y = data["dev"]
        
        if do_proj:
            with open("acl/projs/{}-{}.pickle".format(pair[0], pair[1]), "rb") as f:
                P = pickle.load(f)["P"]
            
        train_x_proj = train_x@P
        dev_x_proj = dev_x@P
        pair_str = pair[0]+ "-" + pair[1]
        entropy = get_entropy(dev_z)
        
        for i in range(num_clfs):
            prof_clf =  load_prof_clf(pair, ind=i)
            probs_train = prof_clf.predict_proba(train_x_proj)
            probs_dev = prof_clf.predict_proba(dev_x_proj)
            
            gender_clf = init_classifier()
            gender_clf.fit(probs_train, train_z)
            loss = sklearn.metrics.log_loss(dev_z, gender_clf.predict_proba(probs_dev))
            print(pair_str, i, entropy - loss)
            
            pair2vinfo_hard_before[pair_str][i] = dict()
            
            for eps in epsilons:
                
                hard_labels_train = np.array([[1-eps,eps] if p[1] > 0.5 else [eps,1-eps] for p in probs_train])
                hard_labels_dev = np.array([[1-eps,eps] if p[1] > 0.5 else [eps,1-eps] for p in probs_dev])
                gender_clf = init_classifier()
                gender_clf.fit(hard_labels_train, train_z)
                loss = sklearn.metrics.log_loss(dev_z, gender_clf.predict_proba(hard_labels_dev))
                print("hard ", eps, pair_str, i,  entropy - loss)
                #pair2vinfo_hard[pair_str][i][cutoff] = entropy - loss      
        print("=========================")

professor-attorney 0 0.0008130818692644315
hard  0.55 professor-attorney 0 -0.002193503156774246
hard  0.6 professor-attorney 0 -0.0013127430028635478
hard  0.7 professor-attorney 0 -5.8606770173885714e-05
hard  0.8 professor-attorney 0 -0.0008075338696763579
hard  0.9 professor-attorney 0 -0.0003585057574706507
hard  0.99 professor-attorney 0 -0.0010201818975216215
professor-attorney 1 -0.0002537870566589051
hard  0.55 professor-attorney 1 -0.0004858139365414571
hard  0.6 professor-attorney 1 -0.0001614055739520559
hard  0.7 professor-attorney 1 7.611056328915566e-05
hard  0.8 professor-attorney 1 -0.0002064685800031718
hard  0.9 professor-attorney 1 -0.00044448571443855833
hard  0.99 professor-attorney 1 -0.0004956620372327247
professor-attorney 2 -0.00018721601070958993
hard  0.55 professor-attorney 2 2.467539664019469e-05
hard  0.6 professor-attorney 2 -0.00047493907741535946
hard  0.7 professor-attorney 2 7.644594053624054e-05
hard  0.8 professor-attorney 2 3.146408611465201e-05
h

## Train composition classifier

In [6]:
def train_single_hidden_neuron_net(X, z, nonlinearity = True, hidden_neurons = 2, sigmoid=False, batch_size=2048,
                                  verbose=False):

    # create an MLP with a single sigmoid neuron
    
    if nonlinearity:
        if sigmoid:
            net = torch.nn.Sequential(torch.nn.Linear(X.shape[1], 1,bias=False), torch.nn.Sigmoid(),
                             torch.nn.Linear(1, 1, bias=True))
        else:
            net = torch.nn.Sequential(torch.nn.Linear(X.shape[1], hidden_neurons), torch.nn.Softmax(dim=1),
                             torch.nn.Linear(hidden_neurons, 1))
    else:
        net = torch.nn.Sequential(torch.nn.Linear(X.shape[1], 1), torch.nn.Linear(1,1))
        
    X_torch = torch.tensor(X).float()
    z_torch = torch.tensor(z).float()

    optimizer = torch.optim.Adam(net.parameters())
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for t in range(25000):
        optimizer.zero_grad()

        batch_indices = torch.randint(0, X.shape[0], (batch_size,))
        z_pred_batch = net(X_torch[batch_indices])
        z_batch = z_torch[batch_indices]        
        loss = loss_fn(z_pred_batch.squeeze(), z_batch)
        loss.backward()
        optimizer.step()
        
        if t % 2000 == 0 and verbose:
            print(loss.detach().cpu().numpy().item())
        
    return net, loss.detach().cpu().numpy().item()

def get_loss_value(net, X, z):
    X_torch = torch.tensor(X).float()
    z_torch = torch.tensor(z).int()
    y_pred = net(X_torch)
    loss = torch.nn.BCEWithLogitsLoss()(y_pred, z_torch)
    return loss.item().detach().cpu().numpy()

def get_classification_accuracy(net, X, z):
    # calculate the accuracy of the sigmoid network

    X_torch = torch.tensor(X).float()
    z_torch = torch.tensor(z).int()
    y_pred = (net(X_torch) > 0).int()
    acc = (y_pred.detach().cpu().squeeze().int().numpy() == z).astype(int).mean()
    return acc

def build_equivalent_linear_model(net,X):
    
    if net[0].weight.shape[0] == 2:
        alpha = (net[0].weight[0]-net[0].weight[1]).detach().clone().detach().cpu().numpy()
        print(net[2].weight)
        beta = (net[2].weight[0][0]-net[0].weight[0][1]).detach().clone().detach().cpu().numpy()
    else:
        alpha = net[0].weight.detach().clone().detach().cpu().numpy()
        beta = net[2].weight.detach().clone().detach().cpu().numpy()
        
    threshold = 0.5
    
    gamma = net[2].bias.detach().clone().detach().cpu().numpy()
    
    model = torch.nn.Linear(X.shape[1], 1)
    model.weight.data = torch.tensor(alpha).float()
    D = -(np.log((1-threshold)/threshold) + gamma)/beta
    new_threshold = torch.tensor(-np.log((1-D)/D)).float()
    model.bias.data = -new_threshold 
    return model

with open("acl/interim/pair2genderclf.pickle", "rb") as f:
    pair2genderclf = pickle.load(f)
    